In [ ]:
import os
os.environ["http_proxy"]="127.0.0.1:7890"
os.environ["https_proxy"]="127.0.0.1:7890"

In [ ]:
from openai import OpenAI

# 官方API
# client = OpenAI()

# 第三方低价API
api_key = ""
api_base = ""
client = OpenAI(api_key=api_key, base_url=api_base)

In [ ]:
import telebot

BOT_TOKEN = ""
bot = telebot.TeleBot(BOT_TOKEN)

In [ ]:
from pinecone import Pinecone, ServerlessSpec

PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "telebot"
index = pc.Index(index_name)

# /crawl

In [ ]:
@bot.message_handler(commands=['crawl'])
def initiate_crawl(message):
    text = "您好！在爬取之前您需要回答以下问题：\n1.您需要爬取的文章的内容与什么有关？\n2.您需要爬取多少篇文章？\n3.您是否需要限制爬取的文章为几几年之后发表的？如果需要限制，请显式指出年份信息。\n4.您需要在中国知网还是谷歌学术查询论文？\n注意：\n1.请使用英文进行说明。\n2.如果不限制年份则默认从2000年开始。如果不限制查询数量，则默认10。\n3.建议查询不超过20篇文章，因为爬取非常耗费时间。"
    sent_msg = bot.send_message(message.chat.id, text, parse_mode="Markdown")
    bot.register_next_step_handler(sent_msg, process_query)

In [ ]:
from pydantic import BaseModel, ValidationError

response_format_extract_keywords_and_fields_and_as_ylo_and_number_and_source = {
    "type": "json_schema",
    "json_schema": {
        "name": "response_format_extract_keywords_fields_as_ylo_number_source",
        "schema": {
            "type": "object",
            "properties": {
                "keywords_for_searching":{
                    "type": "string",
                    "description": "从用户请求中提取用于查询文章的关键字。例子: User request: 我需要爬取26篇有关全球胜任力的文章，并且最好都是2020年之后发表的。最好从谷歌学术上爬取。 - Extracted keywords: 全球胜任力"
                },
                "fields_for_searching":{
                    "type": "string",
                    "description": "从用户请求中判断用户需要查询的是哪一个领域的文章。例子: User request: 我需要爬取26篇有关全球胜任力的文章，并且最好都是2020年之后发表的。最好从谷歌学术上爬取。 - Extracted fields: Eduction",
                    "enum": ["Education", "Computer_Science", "Medicine", "Literature", "Other"]
                },
                "as_ylo_for_searching": {
                    "type": "string",
                    "description": "从用户请求中提取的年份信息，提取结果必须为数字。例子: User request: 我需要爬取26篇有关全球胜任力的文章，并且最好都是2020年之后发表的。最好从谷歌学术上爬取。 - Extracted as_ylo: 2020"
                },
                "number_for_searching": {
                    "type": "string",
                    "description": "从用户请求中提取用户需要爬取多少篇文章，提取结果必须为数字。例子: User request: 我需要爬取26篇有关全球胜任力的文章，并且最好都是2020年之后发表的。最好从谷歌学术上爬取。 - Extracted number: 26"
                },
                "source_for_searching": {
                    "type": "string",
                    "description": "从用户请求中提取用户想要查询论文的来源。例子: User request: 我需要爬取26篇有关全球胜任力的文章，并且最好都是2020年之后发表的。最好从谷歌学术上爬取。 - Extracted source: Google_Scholar",
                    "enum": ["CNKI", "Google_Scholar"]
                }
            },
            "required": ["keywords_for_searching", "fields_for_searching", "as_ylo_for_searching", "number_for_searching", "source_for_searching"],
            "additionalProperties": False
        },
        "strict": True
    }
}

class Keywords_and_Fields_and_As_Ylo_and_Number_and_Source(BaseModel):
    keywords_for_searching: str
    fields_for_searching: str
    as_ylo_for_searching: str
    number_for_searching: str
    source_for_searching: str

response_format_final_fields = {
    "type": "json_schema",
    "json_schema": {
        "name": "response_format_final_fields",
        "schema": {
            "type": "object",
            "properties": {
                "final_fields":{
                    "type": "string",
                    "description": "参考我们训练的文本分类模型的预测和gpt-4o-mini-2024-07-18的预测，并根据原始的user_request，对用户需要查询的是哪一个领域的文章进行最终预测。",
                    "enum": ["Education", "Computer_Science", "Medicine", "Literature", "Other"]
                },
            },
            "required": ["final_fields"],
            "additionalProperties": False
        },
        "strict": True
    }
}

class Final_Fields(BaseModel):
    final_fields: str

In [ ]:
import time
from requests.exceptions import SSLError

from request_processor import request_processor

def process_query(message): 
    user_request = message.text # 用户请求类似 "I need to crawl 26 articles about global competency, preferably all published after 2020. Better to crawl it from Google Scholar."

    messages_to_model=[
        {"role": "system", "content": "You are an expert at structured data extraction. You will be given unstructured text from a user request and should convert it into the given structure."},
        {"role": "user", "content": user_request}
    ]
    response = client.chat.completions.create(model="gpt-4o-mini-2024-07-18", messages=messages_to_model, response_format=response_format_extract_keywords_and_fields_and_as_ylo_and_number_and_source)
    
    try:
        keywords_and_fields_and_as_ylo_and_number_and_source = Keywords_and_Fields_and_As_Ylo_and_Number_and_Source.parse_raw(response.choices[0].message.content)
        # print("@@@@@@", keywords_and_fields_and_as_ylo_and_number)
        keywords_and_fields_and_as_ylo_and_number_and_source_dict = keywords_and_fields_and_as_ylo_and_number_and_source.dict()
        keywords = keywords_and_fields_and_as_ylo_and_number_and_source_dict["keywords_for_searching"] # str
        fields = keywords_and_fields_and_as_ylo_and_number_and_source_dict["fields_for_searching"] # str
        as_ylo = keywords_and_fields_and_as_ylo_and_number_and_source_dict["as_ylo_for_searching"] # str
        number = keywords_and_fields_and_as_ylo_and_number_and_source_dict["number_for_searching"] # str
        source = keywords_and_fields_and_as_ylo_and_number_and_source_dict["source_for_searching"] # str
        # print("******", keywords, fields, as_ylo, number)
    except ValidationError as e:
        print("### Keywords_and_Fields_and_As_Ylo_and_Number.parse_raw(response.choices[0].message.content) 出现错误 ###", e.json())

    ########################## gpt-4o-mini + 文本分类模型 -> gpt-4o 综合判断 ##########################
    predicted_field, predicted_prob = request_processor(user_request) # 使用我们的文本分类模型判断用户请求属于哪一个fields
    predicted_field, predicted_prob = str(predicted_field), str(predicted_prob)
    
    user_messages_for_final_fields = f"用户请求为：{user_request}。我们训练的文本分类模型判断用户请求属于：{predicted_field}。gpt-4o-mini判断用户请求属于：{fields}"
    
    # 将gpt-4o-mini-2024-07-18判断的fields和文本分类模型判断的fields发送给gpt-4o-2024-08-06(使用此版本仅为了为了适应json格式的输出)，让gpt-4o-2024-08-06最终判断一下fields
    messages_for_final_fields = [
        {"role": "system", "content": "You are an expert in text classification and decision-making. You will be provided with a user request, along with predictions from two models regarding the likely field or category of the request. Your task is to analyze the provided information and make a final, authoritative decision on which category the user request belongs to. Consider the input from both models carefully before making your decision."},
        {"role": "user", "content": user_messages_for_final_fields}
    ]
    response = client.chat.completions.create(model="gpt-4o-2024-08-06", messages=messages_for_final_fields, response_format=response_format_final_fields)

    try:
        final_fields = Final_Fields.parse_raw(response.choices[0].message.content)
        final_fields_dict = final_fields.dict()
        final_fields = final_fields_dict["final_fields"]
    except ValidationError as e:
        print("### Final_Fields.parse_raw(response.choices[0].message.content) 出现错误 ###", e.json())
    ########################## gpt-4o-mini + 文本分类模型 -> gpt-4o 综合判断 ##########################
    
    # 如果是无所谓数量和年份，则返回为空，我们必须处理这种情况
    if not as_ylo:  # 检查 as_ylo 是否为 None 或者空字符串
        # print("as_ylo is None or empty, 默认使用2000")
        as_ylo = "2000"
    if not number:  # 检查 number 是否为 None 或者空字符串
        # print("number is None or empty, 默认使用10")
        number = "10"

    text = f"我们判断您需要从{source}查询{final_fields}领域的，与{keywords}有关的从{as_ylo}开始的{number}篇文章。请问是否正确，如果正确我们将开始查询，如果错误，请即使指出！"
    # 为防止网络问题导致的问题，使用try提高下容错
    try:
        sent_msg = bot.send_message(message.chat.id, text, parse_mode=None)
    except SSLError as e:
        print(f"SSL error occurred when bot.send_message(): {e}. Retrying...")
        time.sleep(5)  # 等待几秒后重试
        sent_msg = bot.send_message(message.chat.id, text, parse_mode=None)

    bot.register_next_step_handler(sent_msg, confirm_and_show_extracted_information, keywords, final_fields, as_ylo, number, source, user_request, fields, predicted_field) # gpt4omini - fields; 文本分类模型 - predicted_field

In [ ]:
# 根据用户反馈判断提取出来的信息是否出错
response_format_is_correct = {
    "type": "json_schema",
    "json_schema": {
        "name": "response_format_is_correct",
        "schema": {
            "type": "object",
            "properties": {
                "is_correct": {
                    "type": "boolean",
                    "description": "Indicates whether the user approves the extracted information.",
                    "enum": [True, False]
                },
            },
            "required": ["is_correct"],
            "additionalProperties": False
        },
        "strict": True
    }
}

class UserApproval(BaseModel):
    is_correct: bool

# 加入提取出来的信息出现了错误，则根据提取出来的信息和用户反馈对错误信息进行修正
response_format_true_info = {
    "type": "json_schema",
    "json_schema": {
        "name": "response_format_true_real_info",
        "schema": {
            "type": "object",
            "properties": {
                "true_keywords": {
                    "type": "string",
                    "description": "The confirmed or corrected keywords based on the extracted information and the user's feedback."
                },
                "true_fields": {
                    "type": "string",
                    "description": "The confirmed or corrected field based on the extracted information and the user's feedback.",
                    "enum": ["Education", "Computer_Science", "Medicine", "Literature", "Other"]
                },
                "true_as_ylo": {
                    "type": "string",
                    "description": "The confirmed or corrected as_ylo (year lower bound) based on the extracted information and the user's feedback."
                },
                "true_number": {
                    "type": "string",
                    "description": "The confirmed or corrected number of articles to crawl based on the extracted information and the user's feedback."
                },
                "true_source": {
                    "type": "string",
                    "description": "The confirmed or corrected source (e.g., CNKI, Google Scholar) based on the extracted information and the user's feedback.",
                    "enum": ["CNKI", "Google_Scholar"]
                }
            },
            "required": ["true_keywords", "true_fields", "true_as_ylo", "true_number", "true_source"],
            "additionalProperties": False
        },
        "strict": True
    }
}

class TrueInfo(BaseModel):
    true_keywords: str
    true_fields: str
    true_as_ylo: str
    true_number: str
    true_source: str

In [ ]:
from AppendRowtoCSV import append_row_to_csv

def confirm_and_show_extracted_information(message, keywords, final_fields, as_ylo, number, source, user_request, fields, predicted_field): # gpt4omini - fields; 文本分类模型 - predicted_field
    message_text = message.text  # 示例：1.判断正确的。2.不对，是有关计算机领域！

    messages_to_model=[
        {"role": "system", "content": "You are an expert in information validation. The user has been presented with extracted information (keywords, fields, etc.). Your task is to analyze the user's feedback and determine whether they confirm the correctness of the presented information or not."},
        {"role": "user", "content": message_text}
    ]
    response = client.chat.completions.create(model="gpt-4o-2024-08-06", messages=messages_to_model, response_format=response_format_is_correct)

    try:
        is_correct = UserApproval.parse_raw(response.choices[0].message.content)
        is_correct_dict = is_correct.dict()
        is_correct = is_correct_dict["is_correct"]
    except ValidationError as e:
        print("### UserApproval.parse_raw(response.choices[0].message.content) 出现错误 ###", e.json())
        return

    # 如果正确 ###############################################################################################################################
    if is_correct: 
        
        if predicted_field != final_fields: # 如果用户认为final_fields正确，但我们的文本分类模型判断错了，则需要记录一下
            csv_file = f'data/data_misclassified_user_requests.csv'
            new_row = {'Text': user_request, 'Field_true': final_fields, "Field_predictedbyModel": predicted_field}
            append_row_to_csv(csv_file, new_row)
        if fields != final_fields: # 如果用户认为final_fields正确，但gpt-4o-mini判断错了，则也需要记录一下 - 暂时没写 - 暂时规划：判断错的信息作为prompt返回给gpt-4o-mini
            pass
            
        bot.send_message(message.chat.id, "Very Good! 开始查询，请稍后。", parse_mode=None)
        
        #################################### 爬取 ####################################
        df_crawled = crawl_details(source, keywords, final_fields, as_ylo, number)
        #################################### 爬取 ####################################
        
        bot.send_message(message.chat.id, "查询完成，请等待模型总结查询到的内容。", parse_mode=None)
        ###### 展示搜索到的所有结果 ######
        crawled_text = convert_csv_to_text(df_crawled, source)
        # print("!!!", crawled_text)
        # 将查询的到结果进行简化
        if crawled_text == "网站暂时无法爬取，请稍后！": # 与 convert_csv_to_text() 中对应
            bot.send_message(message.chat.id, "网站暂时无法爬取，请稍后！", parse_mode=None)
        else:
            summarized_text = summarize_and_format_articles(crawled_text, source)
            # print("@@@", summarized_text)
            # 将summarized_text拆分成最大长度为4096的chunk进行发送
            text_chunks = split_text_into_chunks(summarized_text)
            for chunk in text_chunks:
                bot.send_message(message.chat.id, chunk, parse_mode="Markdown")
            sent_msg = bot.send_message(message.chat.id, "以上是本次全部的查询内容。本次查询的数据储存在临时数据库中，是否需要合并到永久数据库中？\n合并可用于类似情况的推荐。", parse_mode="Markdown")
            bot.register_next_step_handler(sent_msg, merge_data, final_fields)
    
    # 如果错误 ###############################################################################################################################
    else: 
        text = f"我们提取出来的信息为：1.用于查询的关键字：{keywords}；2.查询属于的领域：{final_fields}；3.文章的发表年份需要在{as_ylo}年以后；4.需要爬取{number}篇文章。5.需要从{source}查询文章。用户对我们提取的信息的反馈为：{message_text}"
        messages_to_model = [
            {"role": "system", "content": "You are an expert in information validation. Based on the user's feedback, correct only the parts of the extracted information that are incorrect and confirm the others."},
            {"role": "user", "content": text}
        ]
        response = client.chat.completions.create(model="gpt-4o-2024-08-06", messages=messages_to_model, response_format=response_format_true_info)

        try:
            true_info = TrueInfo.parse_raw(response.choices[0].message.content)
            true_info_dict = true_info.dict()
            keywords = true_info_dict["true_keywords"] # str
            final_fields = true_info_dict["true_fields"] # str
            as_ylo = true_info_dict["true_as_ylo"] # str
            number = true_info_dict["true_number"] # str
            source = true_info_dict["true_source"] # str
            bot.send_message(message.chat.id, f"非常抱歉造成错误，我们已经记录了错误信息！Corrected Information: keywords={keywords}, fields={final_fields}, as_ylo={as_ylo}, number={number}, source={source}。开始查询，请稍后！", parse_mode=None)
        except ValidationError as e:
            print("### TrueInfo.parse_raw(response.choices[0].message.content) 出现错误 ###", e.json())
            return

        if predicted_field != final_fields: # 如果用户认为final_fields正确，但我们的文本分类模型判断错了，则需要记录一下
            csv_file = f'data/data_misclassified_user_requests.csv'
            new_row = {'Text': user_request, 'Field_true': final_fields, "Field_predictedbyModel": predicted_field}
            append_row_to_csv(csv_file, new_row)
        if fields != final_fields: # 如果用户认为final_fields正确，但gpt-4o-mini判断错了，则也需要记录一下 - 暂时没写 - 暂时规划：判断错的信息作为prompt返回给gpt-4o-mini
            pass

        #################################### 爬取 ####################################
        df_crawled = crawl_details(source, keywords, final_fields, as_ylo, number)
        #################################### 爬取 ####################################
        
        bot.send_message(message.chat.id, "查询完成，请等待模型总结查询到的内容。", parse_mode=None)
        ###### 展示搜索到的所有结果 ######
        crawled_text = convert_csv_to_text(df_crawled, source)
        # print("!!!", crawled_text)
        # 将查询的到结果进行简化
        if crawled_text == "网站暂时无法爬取，请稍后！": # 与 convert_csv_to_text() 中对应
            bot.send_message(message.chat.id, "网站暂时无法爬取，请稍后！", parse_mode=None)
        else:
            summarized_text = summarize_and_format_articles(crawled_text, source)
            # print("@@@", summarized_text)
            # 将summarized_text拆分成最大长度为4096的chunk进行发送
            text_chunks = split_text_into_chunks(summarized_text)
            for chunk in text_chunks:
                bot.send_message(message.chat.id, chunk, parse_mode="Markdown")
            sent_msg = bot.send_message(message.chat.id, "以上是本次全部的查询内容。本次查询的数据储存在临时数据库中，是否需要合并到永久数据库中？\n合并可用于类似情况的推荐。", parse_mode="Markdown")
            bot.register_next_step_handler(sent_msg, merge_data, final_fields)

In [ ]:
import re

def q_processor(input_string):
    # 删除所有标点符号
    cleaned_string = re.sub(r'[^\w\s]', '', input_string)  # \w匹配字母数字字符，\s匹配空白字符
    # 将左右的空格转换为'+'
    cleaned_string = re.sub(r'\s+', '+', cleaned_string.strip())  
    return cleaned_string

from crawler.GoogleScholar import fetch_and_save_results_temp
from crawler.CNKI import get_page_text,parse_page_text
    
def crawl_details(source, keywords, final_fields, as_ylo, number):
    temp_save_path = f"data/data_crawled/database_temporary/{final_fields}.csv"  # 保存路径
    q = q_processor(keywords)
    if source == "Google_Scholar":
        start = int(number) // 10 # 判断 number 是否可以被 10 整除
        if int(number) % 10 != 0:
            start += 1
        # print("###", q, as_ylo, start)
        df_crawled = fetch_and_save_results_temp(temp_save_path, q, int(as_ylo), start)
        # print("###", df_crawled)
        return df_crawled
    elif source == "CNKI":
        num_pages = int(number) // 20 + 1 # number除以20的整数部分 + 1
        page_text = get_page_text("http://search.cnki.com.cn/Search/ListResult", q, num_pages)
        df_crawled = parse_page_text(page_text, int(number), temp_save_path)
        # print("###", df_crawled)
        return df_crawled

In [ ]:
import pandas as pd

def convert_csv_to_text(df, source):
    if df is None or df.empty:
        return "网站暂时无法爬取，请稍后！"

    total_rows = len(df)
    articles = []

    for i in range(total_rows):
        article = f"搜索到的第{i+1}篇文章：\n"

        if source == "Google_Scholar":
            title = df.get('Title', pd.Series([None]*total_rows)).iloc[i]
            href = df.get('href', pd.Series([None]*total_rows)).iloc[i]
            basic_info = df.get('basic_info', pd.Series([None]*total_rows)).iloc[i]
            concise_abstract = df.get('Abstract', pd.Series([None]*total_rows)).iloc[i]
            full_abstract = df.get('full_abstract', pd.Series([None]*total_rows)).iloc[i]

            # 按需添加非空字段
            if pd.notna(title):
                article += f"Title: {title}\n"
            if pd.notna(href):
                article += f"Href: {href}\n"
            if pd.notna(basic_info):
                article += f"Basic Information: {basic_info}\n"
            if pd.notna(concise_abstract):
                article += f"Consise Abstract: {concise_abstract}\n"
            if pd.notna(full_abstract) and full_abstract != 'N/A':
                article += f"Full Abstract: {full_abstract}\n"

        elif source == "CNKI":
            title = df.get('Title', pd.Series([None]*total_rows)).iloc[i]
            author = df.get('Author', pd.Series([None]*total_rows)).iloc[i]
            paper_source = df.get('Source', pd.Series([None]*total_rows)).iloc[i]
            paper_type = df.get('Type', pd.Series([None]*total_rows)).iloc[i]
            date = df.get('Date', pd.Series([None]*total_rows)).iloc[i]
            abstract = df.get('Abstract', pd.Series([None]*total_rows)).iloc[i]
            keywords = df.get('Keywords', pd.Series([None]*total_rows)).iloc[i]
            download = df.get('Download', pd.Series([None]*total_rows)).iloc[i]
            citations = df.get('Citations', pd.Series([None]*total_rows)).iloc[i]
            href = df.get('href', pd.Series([None]*total_rows)).iloc[i]

            # 按需添加非空字段
            if pd.notna(title):
                article += f"Title: {title}\n"
            if pd.notna(author):
                article += f"Author: {author}\n"
            if pd.notna(paper_source):
                article += f"Source: {paper_source}\n"
            if pd.notna(paper_type):
                article += f"Type: {paper_type}\n"
            if pd.notna(date):
                article += f"Date: {date}\n"
            if pd.notna(abstract):
                article += f"Abstract: {abstract}\n"
            if pd.notna(keywords):
                article += f"Keywords: {keywords}\n"
            if pd.notna(download):
                article += f"Download: {download}\n"
            if pd.notna(citations):
                article += f"Citations: {citations}\n"
            if pd.notna(href):
                article += f"Link: {href}\n"

        # 确保每篇文章都输出到控制台
        # print("::::::", article)

        # 将每篇文章添加到列表
        articles.append(article)

    # 将所有文章内容合并并返回
    return "\n".join(articles)

In [ ]:
def summarize_and_format_articles(text, source):
    if source == "Google_Scholar":
        system_message_content = (
            "You are an expert in summarizing academic articles. The user will provide you with details of several articles including title, href, "
            "basic information(including authors, year, journal), and abstract(consise abstract is always available, but the full abstract may not be available). "
            "Your task is to generate a concise summary of each article, focusing on key details and simplifying the abstract where possible. "
            "Format the output in the following manner:\n\n"
            "1. **Title of the Article**\n"
            "   - **Author:** Author's Name\n"
            "   - **Year:** Year of Publication\n"
            "   - **Journal:** Journal Name\n"
            "   - **Href:** href\n"
            "   - **Summary:** Simplified abstract or summary of the article.\n\n"
            "Ensure each article follows this structure, and keep the summaries brief and clear."
        )
    elif source == "CNKI":
        system_message_content = (
            "You are an expert in summarizing academic articles from CNKI. The user will provide you with details of several articles including title, author, source (journal), "
            "type of the article, date of publication, abstract, keywords, download and citation counts, and link. Your task is to generate a concise summary of each article, "
            "focusing on key details and simplifying the abstract where possible. Format the output in the following manner:\n\n"
            "1. **Title of the Article**\n"
            "   - **Author:** Author's Name\n"
            "   - **Source (Journal):** Journal Name\n"
            "   - **Type:** Type of the article\n"
            "   - **Date:** Date of Publication\n"
            "   - **Keywords:** Keywords associated with the article\n"
            "   - **Download Count:** Number of downloads\n"
            "   - **Citation Count:** Number of citations\n"
            "   - **Link:** Article link\n"
            "   - **Summary:** Simplified abstract or summary of the article.\n\n"
            "Ensure each article follows this structure, and keep the summaries brief and clear."
        )
    else:
        return "Invalid source specified."

    # Construct messages based on the source
    messages = [
        {"role": "system", "content": system_message_content},
        {"role": "user", "content": text}
    ]
    
    # Call the API to summarize and format the articles
    response = client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    summary = response.choices[0].message.content
    return summary

In [ ]:
# 解决 Bad Request: message is too long 的问题 - Telegram 发送消息时的字符限制是 4096 个字符
def split_text_into_chunks(text, chunk_size=4096):
    chunks = []
    
    while len(text) > chunk_size:
        # 尝试在chunk_size之前找到最后一个空行位置
        split_index = text[:chunk_size].rfind('\n\n')
        
        # 如果没有找到空行，则回退到最后一个单独换行符处
        if split_index == -1:
            split_index = text[:chunk_size].rfind('\n')
        
        # 如果找不到单独的换行符，默认拆分点为chunk_size
        if split_index == -1:
            split_index = chunk_size
        
        chunks.append(text[:split_index].strip())  # 去除末尾多余的空格
        text = text[split_index:].strip()  # 去除开头多余的空格
    
    # 添加最后的剩余部分
    chunks.append(text.strip())
    
    return chunks

In [ ]:
from pydantic import BaseModel, ValidationError

# 定义请求格式
response_format_merge_request_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "merge_request_schema",
        "schema": {
            "type": "object",
            "properties": {
                "merge_to_permanent": {
                    "type": "boolean",
                    "description": "Indicates whether the user wants to merge the temporary database into the permanent database.",
                    "enum": [True, False]
                },
            },
            "required": ["merge_to_permanent"],
            "additionalProperties": False
        },
        "strict": True
    }
}

# 定义 Pydantic 模型
class MergeRequest(BaseModel):
    merge_to_permanent: bool

import os
import pandas as pd

# 合并数据的函数
def merge_data(message, final_fields):
    message_text = message.text  # 类似：1.是的需要合并；2.不需要合并

    messages_to_model = [
        {"role": "system", "content": "You are an expert in database management. The user has a temporary database, and your task is to analyze the user's request and determine whether they want to merge this temporary database into the permanent database."},
        {"role": "user", "content": message_text}
    ]
    response = client.chat.completions.create(model="gpt-4o-mini-2024-07-18", messages=messages_to_model, response_format=response_format_merge_request_schema)

    try:
        merge_request = MergeRequest.parse_raw(response.choices[0].message.content)
        merge_request_dict = merge_request.dict()
        merge_to_permanent = merge_request_dict["merge_to_permanent"]
    except ValidationError as e:
        print("### MergeRequest.parse_raw(response.choices[0].message.content) 出现错误 ###", e.json())
        return

    if merge_to_permanent:  # 需要合并
        temp_file_path = f"data/data_crawled/database_temporary/{final_fields}.csv"
        perm_file_path = f"data/data_crawled/database_persistent/{final_fields}.csv"

        # 检查临时数据库中的文件是否存在
        if not os.path.exists(temp_file_path):
            print(f"### 临时数据库中的文件 {temp_file_path} 不存在 ###")
            return

        # 读取临时数据库中的 CSV 文件
        temp_df = pd.read_csv(temp_file_path)

        # 如果永久数据库中没有对应的 CSV 文件，则直接保存
        if not os.path.exists(perm_file_path):
            temp_df.to_csv(perm_file_path, index=False)
            print(f"### {final_fields}.csv 已合并到永久数据库 ###")
        else:
            # 读取永久数据库中的 CSV 文件
            perm_df = pd.read_csv(perm_file_path)

            # 获取所有的列名（包括 Google Scholar 和 CNKI 的列）
            all_columns = list(set(temp_df.columns).union(set(perm_df.columns)))

            # 将缺失的列填充为 None，保证两个数据框的列一致
            for column in all_columns:
                if column not in temp_df.columns:
                    temp_df[column] = None
                if column not in perm_df.columns:
                    perm_df[column] = None

            # 进行合并并去重
            merged_df = pd.concat([perm_df, temp_df], ignore_index=True)
            merged_df.drop_duplicates(inplace=True)

            # 保存合并后的数据到永久数据库
            merged_df.to_csv(perm_file_path, index=False)
            print(f"### {final_fields}.csv 已与永久数据库中的现有文件合并 ###")

        print("### 数据集已成功合并到永久数据库 ###")
        bot.send_message(message.chat.id, "合并完成！", parse_mode=None)
    else:
        print("### 用户选择不合并临时数据库 ###")
        bot.send_message(message.chat.id, "好的，本次查询内容将不会被合并。", parse_mode=None)

# /recommend

In [ ]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# 假设你已经加载了 df，包含 'Abstract', 'Title', 和 'Href' 列

# 1. 生成嵌入
def create_embeddings(text):
    response = client.embeddings.create(input=text, model='text-embedding-3-large')  # 使用适当的模型
    embeddings = response.data[0].embedding
    return embeddings

# 生成嵌入并存入df新的一列 'Abstract embeddings'
def generate_embeddings_for_dataframe(df, file_path):
    # 检查 'Abstract embeddings' 列是否存在，如果不存在则创建
    if 'Abstract embeddings' not in df.columns:
        df['Abstract embeddings'] = None

    embeddings_updated = False

    # 遍历 'Abstract' 列，只有当 'Abstract embeddings' 为空时才生成嵌入
    for i, abstract in df['Abstract'].items():  # 修改为 items()
        if pd.isna(df.at[i, 'Abstract embeddings']):
            df.at[i, 'Abstract embeddings'] = create_embeddings(abstract)
            embeddings_updated = True  # 标记更新

    # 仅当更新了嵌入时才保存到原文件，避免不必要的I/O操作
    if embeddings_updated:
        df.to_csv(file_path, index=False)
        print(f"### 已更新嵌入并保存到 {file_path} ###")
    else:
        print(f"### 没有新的嵌入需要更新，跳过保存 ###")

    return df

import ast  # 用于将字符串转化为列表

# 处理抽象嵌入列表，确保每个嵌入都是list而非str
def process_abstract_embeddings(abstract_embeddings):
    processed_embeddings = []
    for embedding in abstract_embeddings:
        if isinstance(embedding, str):  # 如果嵌入是字符串格式
            try:
                embedding = ast.literal_eval(embedding)  # 将字符串转为列表
            except (ValueError, SyntaxError):
                print(f"无法解析嵌入：{embedding}")
                continue
        processed_embeddings.append(embedding)
    return processed_embeddings

# 2. 根据用户请求生成嵌入并找到最相似的Top_k论文
def find_similar_papers(request_vector, df, top_k=5):
    abstract_embeddings = df['Abstract embeddings'].tolist()
    
    # 确保所有嵌入是正确的数值列表
    abstract_embeddings = process_abstract_embeddings(abstract_embeddings)
    
    # print(type(abstract_embeddings), type(abstract_embeddings[0]), len(abstract_embeddings)) # 第一次输出:<class 'list'> <class 'list'> 10,第二次输出:<class 'list'> <class 'str'> 10
    
    # 使用 NearestNeighbors 查找最相似的向量
    nbrs = NearestNeighbors(n_neighbors=top_k, algorithm='ball_tree').fit(abstract_embeddings)
    
    distances, indices = nbrs.kneighbors([request_vector])
    
    # 收集最相似的论文 title 和 href
    similar_papers = []
    for index, distance in zip(indices[0], distances[0]):
        similar_papers.append({
            'Title': df['Title'].iloc[index],
            'Href': df['href'].iloc[index],
            'Distance': distance
        })
    
    return similar_papers

In [ ]:
# 3. 实现推荐功能
@bot.message_handler(commands=['recommend'])
def recommend_initialize(message):
    """
    让用户选择领域并提问。
    """
    markup = telebot.types.ReplyKeyboardMarkup(one_time_keyboard=True) # 卧槽,牛逼啊,变成选择题了!
    fields = ['Education', 'Computer_Science', 'Medicine', 'Literature', 'Other']
    for field in fields:
        markup.add(field)
    bot.send_message(message.chat.id, "请选择一个领域:", reply_markup=markup)
    bot.register_next_step_handler(message, handle_fields)

def handle_fields(message):
    selected_field = message.text # 例如:Education
    file_path = f"data/data_crawled/database_persistent/{selected_field}.csv"
    bot.send_message(message.chat.id, "您需要我推荐什么文章?")
    bot.register_next_step_handler(message, recommend_papers, file_path, top_k=3)

def recommend_papers(message, file_path, top_k=5):
    request = message.text
    df = pd.read_csv(file_path)
    # 为用户请求生成嵌入
    request_embedding = create_embeddings(request)

    # 更新 DataFrame 中的嵌入并保存到文件
    df = generate_embeddings_for_dataframe(df, file_path)

    # 找到与用户请求最相似的论文
    similar_papers = find_similar_papers(request_embedding, df, top_k=top_k)
    
    # 组合推荐结果
    total_content = "Here are the top recommended papers based on your request:\n\n"
    for paper in similar_papers:
        total_content += f"Title: {paper['Title']}\n"
        total_content += f"Link: {paper['Href']}\n"
        total_content += f"Similarity Score: {paper['Distance']}\n\n"
    
    # 使用 OpenAI 生成完整的推荐响应（可选）
    messages = [
        {"role": "system", "content": "You are an AI assistant that helps with academic paper recommendations."},
        {"role": "user", "content": total_content}
    ]

    # 生成回复
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    
    bot.send_message(message.chat.id, completion.choices[0].message.content, parse_mode="Markdown")

# /news

In [ ]:
@bot.message_handler(commands=['news'])
def initiate_query_news(message):
    text = "您需要查询什么最新的新闻？"
    sent_msg = bot.send_message(message.chat.id, text, parse_mode="Markdown")
    bot.register_next_step_handler(sent_msg, process_news_query)  # 修改函数名称

In [ ]:
import requests
import json

def query_perplexity(user_request):
    url = "https://api.perplexity.ai/chat/completions"
    
    # 在用户请求后追加一句话请求返回来源
    user_request_with_source = user_request + " Please include the source URL of the news."
    
    payload = {
        "model": "llama-3.1-sonar-small-128k-online",
        "messages": [
            {
                "role": "system",
                "content": "Provide a concise and factual response, with clear and precise information."
            },
            {
                "role": "user",
                "content": user_request_with_source  # 动态使用用户请求，并请求来源
            }
        ],
        "max_tokens": 4000,
        "temperature": 0.2,
        "top_p": 0.9,
        "return_citations": True,  # 启用返回引用
        "search_domain_filter": ["perplexity.ai"],
        "return_images": False,
        "return_related_questions": False,
        "search_recency_filter": "month",
        "top_k": 0,
        "stream": False,
        "presence_penalty": 0,
        "frequency_penalty": 1
    }
    
    headers = {
        "Authorization": "Bearer <token>",  # 替换为你的token
        "Content-Type": "application/json"
    }
    
    response = requests.request("POST", url, json=payload, headers=headers)
    
    # 检查响应状态
    if response.status_code == 200:
        data = json.loads(response.text)
        content = data['choices'][0]['message']['content']
        return content  # 返回API的回答
    else:
        return f"Error: {response.status_code} - {response.text}"  # 返回错误信息

In [ ]:
# 修改函数名称，并处理输入请求
def process_news_query(message): 
    user_request = message.text
    perplexity_answer = query_perplexity(user_request)
    bot.send_message(message.chat.id, perplexity_answer, parse_mode="Markdown")
    end_message = "本次查询结束。如想重新查询，请输入“/query_news”。"
    bot.send_message(message.chat.id, end_message, parse_mode=None)

# /upload

In [ ]:
# 确保保存PDF文件的文件夹存在
base_dir = 'rawPDFs/'
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# 确保所有类别的文件夹存在
fields = ["Education", "Computer_Science", "Medicine", "Literature", "Other"]
for field in fields:
    field_path = os.path.join(base_dir, field)
    if not os.path.exists(field_path):
        os.makedirs(field_path)

In [ ]:
# 处理命令上传文件并指定类别
@bot.message_handler(commands=['upload'])
def handle_upload_command(message):
    # text = "您好！请上传您的文档，并在'添加说明'中显式说明您的文档属于 Eduction, Computer_Science, Medicine, Literature, Other 这五个领域中的哪一个？\n注意：请单个单个上传文档！"
    # sent_msg = bot.send_message(message.chat.id, text, parse_mode="Markdown")
    text = "您好！请上传您的文档，并在“添加说明”中显式说明您的文档属于 Education, Computer_Science, Medicine, Literature, Other 这五个领域中的哪一个？另外，请单个单个上传文档！"
    sent_msg = bot.send_message(message.chat.id, text, parse_mode=None)
    bot.register_next_step_handler(sent_msg, handle_document)

In [ ]:
from pydantic import BaseModel, ValidationError

response_format_extract_fields = {
    "type": "json_schema",
    "json_schema": {
        "name": "response_format_extract_fields",
        "schema": {
            "type": "object",
            "properties": {
                "fields":{
                    "type": "string",
                    "description": "从用户请求中判断用户的文档属于哪一个领域。例子: User request: 我的文档属于计算机领域。 - Extracted fields: Computer_Science",
                    "enum": ["Education", "Computer_Science", "Medicine", "Literature", "Other"]
                }
            },
            "required": ["fields"],
            "additionalProperties": False
        },
        "strict": True
    }
}

class Fields(BaseModel):
    fields: str

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=api_key, base_url=api_base) # 使用第三方低价API

In [ ]:
from langchain.document_loaders import PyPDFLoader # 处理单个PDF
from langchain.document_loaders import PyPDFDirectoryLoader # 处理文件夹下的所有PDF
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_pinecone import PineconeVectorStore

from uuid import uuid4

@bot.message_handler(content_types=['document'])
def handle_document(message):
    if message.caption:  # 用户可能在上传文件时指定了类别
        fields_message = message.caption.strip()
        # print("######", fields_message, "######")
        messages_to_model=[
            {"role": "system", "content": "You are an expert at structured data extraction. You will be given unstructured text from a user request and should convert it into the given structure."},
            {"role": "user", "content": fields_message}
        ]
        response = client.chat.completions.create(model="gpt-4o-mini-2024-07-18", messages=messages_to_model, response_format=response_format_extract_fields)
    
        try:
            fields = Fields.parse_raw(response.choices[0].message.content)
            fields_dict = fields.dict()
            fields = fields_dict["fields"]
        except ValidationError as e:
            print("### Fields.parse_raw(response.choices[0].message.content) 出现错误 ###", e.json())
            
    else: # 如果没有在“添加说明”中显式地指出类别，则默认分到Other
        fields = "Other"

    # 保存文档到对应类别文件夹
    file_info = bot.get_file(message.document.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    save_path = os.path.join(base_dir, fields, message.document.file_name)
    
    with open(save_path, 'wb') as new_file:
        new_file.write(downloaded_file)
    
    bot.reply_to(message, f"文档已保存到 {save_path}！请稍后!我们将切分文档,并将文档上传到Pinecone矢量数据库.")

    # 保存完到本地之后,先处理文档,再将文件上传到Pinecone矢量数据库
    file_path = save_path # 指定要加载的 PDF 文件路径
    loader = PyPDFLoader(file_path) # 创建 PyPDFLoader 实例
    documents = loader.load() # 加载 PDF 文件并转换为文本数据
    splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap  = 50)
    documents_chunks = splitter.split_documents(documents)

    namespace = fields
    vector_store = PineconeVectorStore(index=index, namespace=namespace, embedding=embeddings)
    uuids = [str(uuid4()) for _ in range(len(documents_chunks))]
    vector_store.add_documents(documents=documents_chunks, ids=uuids)

    bot.reply_to(message, f"文档已经被切分且上传到Pinecone矢量数据库!本次操作结束.")

# /rag

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=api_key, base_url=api_base) # 使用第三方低价API

In [ ]:
# 用户输入命令后选择文档类别
@bot.message_handler(commands=['rag'])
def ask_field(message):
    """
    让用户选择领域并提问。
    """
    markup = telebot.types.ReplyKeyboardMarkup(one_time_keyboard=True) # 卧槽,牛逼啊,变成选择题了!
    fields = ['Education', 'Computer_Science', 'Medicine', 'Literature', 'Other']
    for field in fields:
        markup.add(field)
    bot.send_message(message.chat.id, "请选择一个领域:", reply_markup=markup)
    bot.register_next_step_handler(message, get_user_question)

In [ ]:
def get_user_question(message):
    """
    获取用户的问题并开始处理查询。
    """
    selected_field = message.text
    bot.send_message(message.chat.id, "请输入您的问题:")
    bot.register_next_step_handler(message, lambda m: process_query(m, selected_field))

def process_query(message, field):
    """
    处理用户查询：在Pinecone中搜索相似内容并生成答案。
    """
    user_question = message.text
    namespace = field
    results = search_similar_documents(user_question, namespace)
    top_k_results = [res.page_content for res, _ in results]
    
    # 将用户问题与检索结果一同传给OpenAI生成最终答案
    final_answer = generate_answer(user_question, top_k_results)
    
    # 返回结果给用户
    bot.send_message(message.chat.id, final_answer, parse_mode="Markdown")

from langchain_pinecone import PineconeVectorStore

def search_similar_documents(query, namespace, top_k=5):
    """
    在Pinecone的特定namespace中查找与query最相似的文档内容。
    """
    vector_store = PineconeVectorStore(index=index, namespace=namespace, embedding=embeddings)
    results = vector_store.similarity_search_with_score(query, k=top_k)
    return results

def generate_answer(user_question, documents):
    """
    将用户问题和检索到的文档内容传入OpenAI模型生成答案。
    """
    context = "\n".join(documents)
    user_prompt = f"用户问题: {user_question}\n相关文档:\n{context}\n请根据这些内容回答问题："

    # system prompt 用于指导模型回答
    system_prompt = (
        "你是一个帮助用户回答基于文档内容的问题的助手。"
        "当用户提出问题时，你应该根据提供的相关文档中的信息进行回答。"
        "如果文档中包含答案，尽可能引用文档内容并确保准确性。"
        "如果文档中没有明确的答案，请基于你对文档的理解提供推断，但避免过度猜测。"
    )
    
    messages_to_model = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # 使用OpenAI生成最终答案
    completion = client.chat.completions.create(model="gpt-4o-mini", messages=messages_to_model)
    # print(user_prompt)
    return completion.choices[0].message.content

# 非命令消息

In [ ]:
import json

# 处理所有非命令的消息
@bot.message_handler(func=lambda message: not message.text.startswith('/'))
def echo_all(message):
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},  
                {"role": "user", "content": message.text}
            ]
        )
        bot_reply = completion.choices[0].message.content
        bot.reply_to(message, bot_reply)

    except Exception as e:
        bot.reply_to(message, "出错了，请稍后再试！")

# 启动

In [ ]:
# 启动Bot
# bot.infinity_polling()
bot.polling()